# Яндекс Недвижимость

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [285]:
import pandas as pd
import re
import random
import math
import numpy as np
from IPython.core.display_functions import display

In [286]:
import requests
from api_key import api_key
import json
from geopy import distance

In [287]:
pd.__version__

'1.4.2'

In [288]:
# set max columns to none
pd.set_option("display.max_columns", None)

In [378]:
pulkovo_long_lat = (30.270441, 59.799816)
hares_island_long_lat = (30.321948, 59.951103)

if type of location not set assumming that its town

In [290]:
def name_n_type(text):
        whole_words = text.split()
        type_list = []
        name_list = []
        name_loc = " "
        type_loc = " "
        
        # regex
        pattern = '[А-ЯЁ]+[а-яё]'
        
        for word in whole_words:
            
            # searching pattern
            if re.search(pattern, word):
                name_list.append(word)
            else:
                type_list.append(word)
        
        name_loc = name_loc.join(name_list)
        
        if len(type_list) == 0:
            type_loc = 'город'
        else:
            type_loc = type_loc.join(type_list)
        
        return name_loc, type_loc

In [291]:
def correct_name(to_correct):
    if 'поселок' in to_correct:
        to_correct = to_correct.replace('поселок', 'посёлок')
    return to_correct

In [292]:
def get_length_2_arprt(
        locality_name,
        locality_type,
        region='ленинградская область'):

    address = (region, locality_type, locality_name)
    address = ', '.join(address)
    maps_url = 'https://geocode-maps.yandex.ru/1.x/'
    keys = {'apikey': api_key,
            'geocode': address,
            'format': 'json',
            'results': '1',
            }

    row_data = requests.get(maps_url, params=keys)
    parsed = json.loads(row_data.text)
    locality_position = (json.dumps(
        parsed['response']['GeoObjectCollection']
        ['featureMember'][0]['GeoObject']['Point']['pos']
        , indent=4, ensure_ascii=False
    )
    )
    locality_position = locality_position.replace('\"', '')
    long, lat = locality_position.split(' ')
    locality_long_lat = (float(long), float(lat))
    dist_in_m = distance.distance(
        locality_long_lat,
        pulkovo_long_lat
    ).m
    return (float(round(dist_in_m)))

In [293]:
real_estate_data = pd.read_csv('./datasets/real_estate_data.csv', sep='\t')

In [294]:
display(real_estate_data['balcony'].unique())

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

In [295]:
display(real_estate_data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,False,False,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,False,False,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,False,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,False,False,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


`'cityCenters_nearest'` → convert to snake case

In [296]:
real_estate_data.rename(columns={'cityCenters_nearest': 'city_centers_nearest'}, inplace=True)

In [297]:
display(real_estate_data['is_apartment'].unique())

array([nan, False, True], dtype=object)

In [298]:
display(real_estate_data['balcony'].unique())

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

<div class="alert alert-info" role="alert">
    <p class="mb-0">Как получить <code>list</code> колонок с <code>NaN</code> нашёл на <code>stackoverflow</code> 👌</p>
</div>

In [299]:
display(
    real_estate_data.columns[real_estate_data.isna().any()].tolist()
)

['ceiling_height',
 'floors_total',
 'living_area',
 'is_apartment',
 'kitchen_area',
 'balcony',
 'locality_name',
 'airports_nearest',
 'city_centers_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

Добро, вот с этими колонками нужно разобраться в них пропуски — `NaN`

In [300]:
display(real_estate_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

In [301]:

for column in real_estate_data.columns:
    count_of_nan = len(real_estate_data.loc[real_estate_data[column].isna()][column])
    if count_of_nan > 0:
        print(column, end=' NaN = ')
        print(count_of_nan)
        print(real_estate_data[column].describe(), end='\n\n')

ceiling_height NaN = 9195
count    14504.000000
mean         2.771499
std          1.261056
min          1.000000
25%          2.520000
50%          2.650000
75%          2.800000
max        100.000000
Name: ceiling_height, dtype: float64

floors_total NaN = 86
count    23613.000000
mean        10.673824
std          6.597173
min          1.000000
25%          5.000000
50%          9.000000
75%         16.000000
max         60.000000
Name: floors_total, dtype: float64

living_area NaN = 1903
count    21796.000000
mean        34.457852
std         22.030445
min          2.000000
25%         18.600000
50%         30.000000
75%         42.300000
max        409.700000
Name: living_area, dtype: float64

is_apartment NaN = 20924
count      2775
unique        2
top       False
freq       2725
Name: is_apartment, dtype: object

kitchen_area NaN = 2278
count    21421.000000
mean        10.569807
std          5.905438
min          1.300000
25%          7.000000
50%          9.100000
75%         

## Шаг 2. Предобработка данных

Определите и изучите пропущенные значения:
Для некоторых пропущенных значений можно предположить логичную замену. Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски правильно заменить на 0. Для других типов данных нет подходящего значения на замену. В этом случае правильно оставить эти значения пустыми. Отсутствие значения — тоже важный сигнал, который не нужно прятать.



Заполните пропуски, где это уместно. Опишите, почему вы решили заполнить пропуски именно в этих столбцах и как выбрали значения.
Укажите причины, которые могли привести к пропускам в данных.

In [302]:
real_estate_data['is_apartment'].fillna(value=False, inplace=True)

In [303]:
display(
    real_estate_data['floor'].loc[(real_estate_data['balcony'].isna())].describe()
)

display(
    real_estate_data['floor'].loc[~(real_estate_data['balcony'].isna())].describe()
)

display(
    real_estate_data['floor']
        .loc[(real_estate_data['balcony'].isna())
            & (real_estate_data['floor'] == 1)]
        .count()
)

count    11519.000000
mean         5.314003
std          4.667232
min          1.000000
25%          2.000000
50%          4.000000
75%          7.000000
max         31.000000
Name: floor, dtype: float64

count    12180.000000
mean         6.439327
std          5.022114
min          1.000000
25%          3.000000
50%          5.000000
75%          9.000000
max         33.000000
Name: floor, dtype: float64

1926

In [304]:
(
    real_estate_data['balcony'].loc[(real_estate_data['balcony'].isna())
                                & (real_estate_data['floor'] == 1)]
                            .fillna(value=0, inplace=True)
)

In [305]:
display(
    real_estate_data['balcony'].unique()
)

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

next step is temp

In [306]:
real_estate_data['balcony'].fillna(value=108, inplace=True)

In [307]:
display(
    real_estate_data['balcony'].unique()
)

array([108.,   2.,   0.,   1.,   5.,   4.,   3.])

In [308]:
real_estate_data['is_apartment'].fillna(value=False, inplace=True)

In [309]:
display(
    real_estate_data.loc[real_estate_data['locality_name'].isna()]
                        .head(10)
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1097,3,8600000.0,81.7,2016-04-15T00:00:00,3,3.55,5.0,50.80,2,False,False,False,8.80,0.0,NaN,23478.0,4258.0,0.0,NaN,0.0,NaN,147.0
2033,6,5398000.0,80.0,2017-05-30T00:00:00,3,NaN,4.0,42.60,2,False,False,False,18.60,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0
2603,20,3351765.0,42.7,2015-09-20T00:00:00,1,NaN,24.0,15.60,3,False,False,False,10.70,0.0,NaN,22041.0,17369.0,0.0,NaN,1.0,374.0,276.0
2632,2,5130593.0,62.4,2015-10-11T00:00:00,2,NaN,24.0,33.10,21,False,False,False,8.20,0.0,NaN,22041.0,17369.0,0.0,NaN,1.0,374.0,256.0
3574,10,4200000.0,46.5,2016-05-28T00:00:00,2,NaN,5.0,30.80,5,False,False,False,6.50,0.0,NaN,27419.0,8127.0,0.0,NaN,1.0,603.0,45.0
4151,17,17600000.0,89.5,2014-12-09T00:00:00,2,3.00,8.0,39.62,7,False,False,False,13.38,0.0,NaN,25054.0,3902.0,1.0,485.0,3.0,722.0,869.0
4189,7,9200000.0,80.0,2015-12-10T00:00:00,3,4.00,4.0,52.30,3,False,False,False,10.40,0.0,NaN,21774.0,3039.0,1.0,690.0,1.0,953.0,223.0
4670,1,5500000.0,83.0,2015-08-14T00:00:00,3,NaN,7.0,NaN,6,False,False,False,NaN,0.0,NaN,26534.0,5382.0,1.0,446.0,1.0,376.0,350.0
5343,19,13540000.0,85.5,2016-01-20T00:00:00,3,NaN,7.0,59.10,5,False,False,False,8.30,4.0,NaN,10556.0,9538.0,1.0,67.0,0.0,NaN,303.0
5707,7,3700000.0,30.0,2016-04-29T00:00:00,1,NaN,24.0,20.00,23,False,False,False,NaN,0.0,NaN,21460.0,16788.0,0.0,NaN,1.0,170.0,49.0


In [310]:
display(
    real_estate_data.columns[real_estate_data.isna().any()].tolist()
)

['ceiling_height',
 'floors_total',
 'living_area',
 'kitchen_area',
 'locality_name',
 'airports_nearest',
 'city_centers_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

    ❌ 'ceiling_height' — сделаем `NaN` медианным значением по району
    ❌ 'floors_total' — сделаем `NaN` медианным значением по району
    ❌ 'living_area' — сделаем `NaN` медианным значением по диапазонам цен и рай
    ✅ 'is_apartment'
    ❌ 'kitchen_area' — если студия то кухня `NaN` проверим, если да то сделаем 0
    ✅ 'balcony'
    ❌ 'locality_name' — сначала удалим неявные дубликаты
    ❌ 'airports_nearest' — проверим по районам, может и нет ближайших аэропортов тогда `NaN` = 0
    ❌ 'city_centers_nearest' — проверим по районам, может и нет ближайших центров города тогда `NaN` = 0
    ❌ 'parks_around3000' — проверим по районам, может и нет ближайших парков тогда `NaN` = 0
    ❌ 'parks_nearest'
    ❌ 'ponds_around3000' — проверим по районам, может и нет ближайших прудов тогда `NaN` = 0
    ❌ 'ponds_nearest'
    ❌ 'days_exposition' — посмотрим как связан столбец с `first_day_exposition`

**Предполагаю**

что стоит заменить пропущенные высоту потолков и количество этажей на медиану районных значений.

### `'locality_name'` — сначала удалим неявные дубликаты

func to reduce dublicates

In [311]:
real_estate_data['locality_name'].fillna('Санкт-Петербург', inplace=True)

In [312]:
print(len(real_estate_data['locality_name'].unique()))
real_estate_data['locality_name'] = real_estate_data['locality_name'].apply(correct_name)
print(len(real_estate_data['locality_name'].unique()))

364
330


type of location:
'деревня'
'садовое товарищество'
'городской посёлок'
'посёлок городского типа'
'коттеджный посёлок'
'посёлок при железнодорожной станции'
'посёлок станции'
'посёлок'
'село'
'садоводческое некоммерческое товарищество'

In [313]:
real_estate_data['locality_name'].describe()

count               23699
unique                330
top       Санкт-Петербург
freq                15770
Name: locality_name, dtype: object

In [314]:
real_estate_data['locality_name'], real_estate_data['locality_type'] = (
    zip(*real_estate_data['locality_name'].apply(name_n_type))
)

In [315]:
display(
    real_estate_data.head()
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_type
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,False,False,False,25.0,108.0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN,город
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,False,False,False,11.0,2.0,Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0,посёлок
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,False,False,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0,город
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,False,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0,город
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,False,False,False,41.0,108.0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0,город


In [316]:
display(
    real_estate_data['locality_type'].unique()
)

array(['город', 'посёлок', 'городской посёлок', 'деревня',
       'посёлок городского типа', 'садовое товарищество', 'село',
       'посёлок городского типа имени', 'посёлок станции',
       'садоводческое некоммерческое товарищество',
       'посёлок 69-й километр', 'посёлок при железнодорожной станции',
       'коттеджный посёлок'], dtype=object)

In [317]:
real_estate_data.loc[real_estate_data['locality_name'] == 'посёлок 69-й километр', 'locality_name'] = '69-й километр'
real_estate_data.loc[real_estate_data['locality_type'] == 'посёлок 69-й километр', 'locality_type'] = 'посёлок'

In [318]:
display(
    real_estate_data['locality_type'].unique()
)

array(['город', 'посёлок', 'городской посёлок', 'деревня',
       'посёлок городского типа', 'садовое товарищество', 'село',
       'посёлок городского типа имени', 'посёлок станции',
       'садоводческое некоммерческое товарищество',
       'посёлок при железнодорожной станции', 'коттеджный посёлок'],
      dtype=object)

In [319]:
locality_type_series = pd.Series(real_estate_data['locality_type'].unique())
display(
    locality_type_series
)

0                                         город
1                                       посёлок
2                             городской посёлок
3                                       деревня
4                       посёлок городского типа
5                          садовое товарищество
6                                          село
7                 посёлок городского типа имени
8                               посёлок станции
9     садоводческое некоммерческое товарищество
10          посёлок при железнодорожной станции
11                           коттеджный посёлок
dtype: object

In [320]:
real_estate_data['locality_type'] = (
    real_estate_data['locality_type'].apply(lambda x: 
        list(locality_type_series[locality_type_series == x].index)[0]
    )
)

In [321]:
display(
    real_estate_data.head()
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_type
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,False,False,False,25.0,108.0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN,0
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,False,False,False,11.0,2.0,Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0,1
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,False,False,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0,0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,False,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0,0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,False,False,False,41.0,108.0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0,0


In [322]:
display(
    real_estate_data['locality_type'].unique()
)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [323]:
real_estate_data = real_estate_data.astype(
    {
        'locality_name': 'category',
        'locality_type': 'category',
    }
)

why there is no inplace parameter in astype

In [324]:
display(
    real_estate_data['balcony'].unique()
)

array([108.,   2.,   0.,   1.,   5.,   4.,   3.])

In [325]:
display(
    real_estate_data.groupby(by=['locality_type'])['balcony'].median()
)

locality_type
0       5.0
1       2.0
2     108.0
3       2.0
4       4.5
5      54.5
6     108.0
7       2.0
8     108.0
9     108.0
10    108.0
11      1.0
Name: balcony, dtype: float64

In [326]:
display(
    real_estate_data.columns
)

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'city_centers_nearest',
       'parks_around3000', 'parks_nearest', 'ponds_around3000',
       'ponds_nearest', 'days_exposition', 'locality_type'],
      dtype='object')

In [327]:
types_na_balcony = (
    list(real_estate_data['locality_type']
             .loc[real_estate_data['balcony'] == 108]
             .unique())
)

for type_loc in types_na_balcony:
    
    balcony_mode_by_type = (
        real_estate_data
            .loc[(real_estate_data['locality_type'] == type_loc)
                & ~(real_estate_data['balcony'] == 108), ['balcony']]
            .mode()
    )
    
    if len(balcony_mode_by_type.index) == 0:
        balcony_mode_by_type = 0
    elif len(balcony_mode_by_type.index) == 1:
        balcony_mode_by_type = balcony_mode_by_type.iloc[0].item()
    else:
        balcony_mode_by_type = balcony_mode_by_type.sample().iloc[0].item()
    
    real_estate_data.loc[(real_estate_data['locality_type'] == type_loc)
                & (real_estate_data['balcony'] == 108), 'balcony'] = (
        real_estate_data
            .loc[(real_estate_data['locality_type'] == type_loc)
                & (real_estate_data['balcony'] == 108), 'balcony']
            .apply(lambda x: balcony_mode_by_type if bool(random.getrandbits(1)) else 0)
    )


In [328]:
display(
    real_estate_data['balcony'].unique()
)

array([1., 2., 0., 5., 4., 3.])

In [329]:
real_estate_data = real_estate_data.astype(
    {'balcony': 'uint8'}
)

In [330]:
display(
    real_estate_data['balcony'].info()
)

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: balcony
Non-Null Count  Dtype
--------------  -----
23699 non-null  uint8
dtypes: uint8(1)
memory usage: 23.3 KB


None

    ❌ 'ceiling_height' — сделаем `NaN` медианным значением по району

In [331]:
types_na_ceiling = (
    list(real_estate_data['locality_type']
             .loc[real_estate_data['ceiling_height'].isna()]
             .unique())
)

for type_ceiling in types_na_ceiling:
    
    ceiling_median_by_type = (
        real_estate_data
            .loc[(real_estate_data['locality_type'] == type_ceiling)
                & ~(real_estate_data['ceiling_height'].isna()), 'ceiling_height']
            .median()
    )
    
    if math.isnan(ceiling_median_by_type):
        ceiling_median_by_type = real_estate_data['ceiling_height'].median()

    real_estate_data.loc[
        real_estate_data['locality_type'] == type_ceiling, 
        'ceiling_height'
    ] = real_estate_data.loc[
            real_estate_data['locality_type'] == type_ceiling, 
            'ceiling_height'
        ].fillna(value=ceiling_median_by_type)

# somehow .fillna(inplace=True) didn't work

In [332]:
real_estate_data['ceiling_height'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: ceiling_height
Non-Null Count  Dtype  
--------------  -----  
23699 non-null  float64
dtypes: float64(1)
memory usage: 185.3 KB


In [333]:
display(
    real_estate_data['ceiling_height'].describe()
)

count    23699.000000
mean         2.721105
std          0.988643
min          1.000000
25%          2.600000
50%          2.650000
75%          2.700000
max        100.000000
Name: ceiling_height, dtype: float64

In [334]:
types_odd_ceiling = (
    list(real_estate_data.loc[
            (real_estate_data['ceiling_height'] <= 2)
            | (real_estate_data['ceiling_height'] > 60)
            | ((real_estate_data['ceiling_height'] > 6)
                & (real_estate_data['ceiling_height'] < 20)
              ), 'locality_type'
        ].unique()
    )
)

for type_ceiling in types_odd_ceiling:
    
    ceiling_median_by_type = (
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_ceiling)
                & (real_estate_data['ceiling_height'] <= 6)
                & (real_estate_data['ceiling_height'] > 2)
            , 'ceiling_height'
        ].median()
    )

    display(ceiling_median_by_type)

    if math.isnan(ceiling_median_by_type):
        ceiling_median_by_type = real_estate_data['ceiling_height'].median()

    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_ceiling)
            & (real_estate_data['ceiling_height'] <= 2)
            | (real_estate_data['ceiling_height'] > 60)
            | ((real_estate_data['ceiling_height'] > 6)
                & (real_estate_data['ceiling_height'] < 20)
              )
            , 'ceiling_height'
    ] = ceiling_median_by_type

2.65

2.65

2.56

2.6

In [335]:
display(
    real_estate_data['ceiling_height'].describe()
)

count    23699.000000
mean         2.715745
std          0.751619
min          2.200000
25%          2.600000
50%          2.650000
75%          2.700000
max         32.000000
Name: ceiling_height, dtype: float64

In [336]:
real_estate_data.loc[
            (real_estate_data['ceiling_height'] >= 20)
            & (real_estate_data['ceiling_height'] <= 60)
            , 'ceiling_height'
] = real_estate_data.loc[
            (real_estate_data['ceiling_height'] >= 20)
            & (real_estate_data['ceiling_height'] <= 60)
            , 'ceiling_height'
    ].apply(lambda x: x / 10)

In [337]:
display(
    real_estate_data['ceiling_height'].describe()
)

count    23699.000000
mean         2.693905
std          0.221764
min          2.000000
25%          2.600000
50%          2.650000
75%          2.700000
max          6.000000
Name: ceiling_height, dtype: float64

    ❌ 'floors_total' — сделаем `NaN` медианным значением по району

In [338]:
display(
    real_estate_data.loc[
        real_estate_data['floors_total'].isna()
    ].groupby(by='floor')['floor'].count()
)

floor
1      3
2      2
3      4
4     10
5      4
6      9
7      4
8      8
9      5
10     3
11     3
12     6
13     3
14     2
15     3
17     1
18     3
19     3
20     2
21     4
22     1
23     1
24     2
Name: floor, dtype: int64

seems missing `'floors_total'` not depends on `'floor'`

In [339]:
display(
    real_estate_data.loc[real_estate_data['floor'] > real_estate_data['floors_total']]
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_type


In [340]:
display(
    real_estate_data.loc[
        real_estate_data['floors_total'].isna()
    ].sample(n=10)
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_type
4617,4,4275062.0,61.20,2017-04-27T00:00:00,2,2.65,NaN,32.00,12,False,False,False,12.20,0,Санкт-Петербург,22041.0,17369.0,0.0,NaN,1.0,374.0,72.0,0
10319,10,3216089.0,35.79,2018-10-01T00:00:00,1,2.65,NaN,10.00,8,False,False,False,NaN,1,Санкт-Петербург,47303.0,25866.0,1.0,251.0,1.0,350.0,145.0,0
13622,3,7650000.0,88.84,2016-05-21T00:00:00,3,2.65,NaN,49.87,21,False,False,False,9.76,0,Санкт-Петербург,35234.0,14674.0,0.0,NaN,1.0,844.0,42.0,0
237,4,2438033.0,28.10,2016-11-23T00:00:00,1,2.65,NaN,20.75,1,False,False,False,NaN,0,Санкт-Петербург,22041.0,17369.0,0.0,NaN,1.0,374.0,251.0,0
9023,5,2900000.0,35.10,2018-07-18T00:00:00,1,2.56,NaN,NaN,11,False,False,False,NaN,0,Новоселье,NaN,NaN,NaN,NaN,NaN,NaN,26.0,2
10568,12,14900000.0,100.00,2018-10-02T00:00:00,3,2.80,NaN,51.00,4,False,False,False,40.00,0,Санкт-Петербург,30766.0,8059.0,0.0,NaN,1.0,213.0,140.0,0
6136,10,5467707.0,58.13,2018-11-04T00:00:00,3,2.65,NaN,31.00,6,False,False,False,NaN,1,Санкт-Петербург,47303.0,25866.0,1.0,251.0,1.0,350.0,110.0,0
5146,10,3085024.0,33.33,2018-10-06T00:00:00,1,2.65,NaN,11.00,12,False,False,False,NaN,1,Санкт-Петербург,47303.0,25865.0,1.0,400.0,1.0,492.0,140.0,0
17721,0,12000000.0,76.50,2017-07-02T00:00:00,2,2.65,NaN,36.30,8,False,False,False,13.80,0,Санкт-Петербург,39393.0,11096.0,1.0,537.0,0.0,NaN,36.0,0
2392,7,5395770.0,53.00,2017-06-29T00:00:00,2,2.65,NaN,29.00,22,False,False,False,9.00,0,Санкт-Петербург,43074.0,10820.0,0.0,NaN,0.0,NaN,48.0,0


In [341]:
types_na_flrs_ttl = (
    list(real_estate_data.loc[
            (real_estate_data['floors_total'].isna())
            , 'locality_type'
        ].unique()
    )
)

print(types_na_flrs_ttl)

for type_flrs_ttl in types_na_flrs_ttl:
    
    flrs_ttl_median_by_type = (
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_flrs_ttl)
            & ~(real_estate_data['floors_total'].isna())
            , 'floors_total'
        ].median()
    )

    print(flrs_ttl_median_by_type)
    
    if math.isnan(flrs_ttl_median_by_type):
        flrs_ttl_median_by_type = real_estate_data['floors_total'].median()
        print('NANы', flrs_ttl_median_by_type)
        
#     real_estate_data.loc[
#         (real_estate_data['locality_type'] == type_flrs_ttl)
#         & (real_estate_data['floors_total'].isna())
#         , 'floors_total'
#     ].fillna(value=flrs_ttl_median_by_type, inplace=True)
    
    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_flrs_ttl)
            & (real_estate_data['floors_total'].isna())
            , 'floors_total'
    ] = flrs_ttl_median_by_type
    
    # real_estate_data.loc[
    #     (real_estate_data['locality_type'] == type_flrs_ttl)
    #     & (real_estate_data['floors_total'].isna())
    #     , 'floors_total'
    # ] = real_estate_data.loc[
    #         (real_estate_data['locality_type'] == type_flrs_ttl)
    #         & (real_estate_data['floors_total'].isna())
    #         , 'floors_total'
    #     ].fillna(value=flrs_ttl_median_by_type)

[0, 1, 2, 4, 3]
9.0
12.0
5.0
5.0
9.0


🤦‍♂️

In [342]:
#     why ↓ it's not working in for loop!?
    
    # real_estate_data.loc[
    #     (real_estate_data['locality_type'] == type_flrs_ttl)
    #     & (real_estate_data['floors_total'].isna())
    #     , 'floors_total'
    # ].fillna(value=flrs_ttl_median_by_type, inplace=True)

In [343]:
display(
        real_estate_data['floors_total'].describe(),
        real_estate_data['floors_total'].info()
)

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: floors_total
Non-Null Count  Dtype  
--------------  -----  
23699 non-null  float64
dtypes: float64(1)
memory usage: 185.3 KB


count    23699.000000
mean        10.668425
std          6.586123
min          1.000000
25%          5.000000
50%          9.000000
75%         16.000000
max         60.000000
Name: floors_total, dtype: float64

None

In [344]:
display(
    real_estate_data.loc[
        real_estate_data['floor'] > real_estate_data['floors_total']
        , 'floor'
    ].describe()
)

count    36.000000
mean     16.166667
std       4.358899
min      10.000000
25%      12.000000
50%      15.000000
75%      20.000000
max      24.000000
Name: floor, dtype: float64

In [345]:
real_estate_data.loc[
    (real_estate_data['floor'] > real_estate_data['floors_total'])
    , 'floor'
] = real_estate_data.loc[
        (real_estate_data['floor'] > real_estate_data['floors_total'])
        , 'floors_total'
    ]

In [346]:
display(
    real_estate_data.loc[real_estate_data['floor'] > real_estate_data['floors_total'], 'floor'].count()
)

0

In [347]:
real_estate_data = real_estate_data.astype(
    {'floors_total': 'uint8',}
)

    ❌ 'living_area' — сделаем `NaN` медианным значением по диапазонам цен и району

In [348]:
# display(
#     real_estate_data.loc[
#         real_estate_data['living_area'].isna()
#         , 'floor'
#     ].count()
# )

In [349]:
display(
    real_estate_data['total_area'].describe()
)

count    23699.000000
mean        60.348651
std         35.654083
min         12.000000
25%         40.000000
50%         52.000000
75%         69.900000
max        900.000000
Name: total_area, dtype: float64

In [350]:
display(
    real_estate_data.loc[
        real_estate_data['total_area'] > 450
    ]
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_type
660,3,49950000.0,483.9,2017-10-04T00:00:00,7,3.20,6,312.5,4,False,False,False,26.4,1,Санкт-Петербург,23079.0,945.0,3.0,461.0,3.0,22.0,391.0,0
3117,19,140000000.0,631.0,2019-03-30T00:00:00,7,2.65,5,NaN,5,False,False,False,60.0,1,Санкт-Петербург,25707.0,4972.0,0.0,NaN,2.0,210.0,NaN,0
4237,1,50000000.0,517.0,2017-10-14T00:00:00,7,2.65,4,332.0,3,False,False,False,22.0,0,Санкт-Петербург,22835.0,1847.0,2.0,477.0,2.0,540.0,NaN,0
5358,20,65000000.0,590.0,2019-02-14T00:00:00,15,3.50,6,409.0,3,False,False,False,100.0,1,Санкт-Петербург,24447.0,4666.0,1.0,767.0,0.0,NaN,14.0,0
5893,3,230000000.0,500.0,2017-05-31T00:00:00,6,2.65,7,NaN,7,False,False,False,40.0,0,Санкт-Петербург,32440.0,5297.0,0.0,NaN,2.0,198.0,50.0,0
6221,8,34000000.0,470.3,2018-01-14T00:00:00,4,2.65,3,248.7,3,False,False,False,40.0,0,Санкт-Петербург,43756.0,15459.0,1.0,852.0,3.0,122.0,179.0,0
6621,20,99000000.0,488.0,2017-04-09T00:00:00,5,2.95,20,216.0,17,False,False,False,50.0,1,Санкт-Петербург,27641.0,4598.0,1.0,646.0,1.0,368.0,351.0,0
8018,20,84000000.0,507.0,2015-12-17T00:00:00,5,4.45,7,301.5,7,True,False,False,45.5,1,Санкт-Петербург,NaN,5735.0,2.0,110.0,0.0,NaN,1078.0,0
9826,1,43185328.0,494.1,2017-05-12T00:00:00,1,2.85,12,NaN,3,False,False,False,NaN,0,Санкт-Петербург,28529.0,6832.0,1.0,592.0,0.0,NaN,5.0,0
12401,20,91500000.0,495.0,2017-06-19T00:00:00,7,4.65,7,347.5,7,False,False,False,25.0,0,Санкт-Петербург,NaN,5735.0,2.0,110.0,0.0,NaN,14.0,0


In [351]:
display(
    real_estate_data['living_area'].describe()
)

count    21796.000000
mean        34.457852
std         22.030445
min          2.000000
25%         18.600000
50%         30.000000
75%         42.300000
max        409.700000
Name: living_area, dtype: float64

In [352]:
types_na_living_area = (
    list(real_estate_data.loc[
            (real_estate_data['living_area'].isna())
            , 'locality_type'
        ].unique()
    )
)

for type_living_area in types_na_living_area:
    
    living_area_median_by_type = (
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & ~(real_estate_data['living_area'].isna())
            , 'living_area'
        ].median()
    )
    
    living_part_median_by_type = round(pd.Series(
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & (real_estate_data['living_area'] > 8)
            & ~(real_estate_data['living_area'].isna())
            , 'living_area'
        ] / real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & (real_estate_data['living_area'] > 8)
            & ~(real_estate_data['living_area'].isna())
            , 'total_area'
        ]
    ).median(), 2)

    if math.isnan(living_area_median_by_type):
        living_area_median_by_type = real_estate_data['living_area'].median()
        print('NANы')
        
    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & ((real_estate_data['living_area'].isna())
            | (real_estate_data['living_area'] < 8))
            , 'living_area'
    ] = real_estate_data.loc[
            (real_estate_data['locality_type'] == type_living_area)
            & ((real_estate_data['living_area'].isna())
            | (real_estate_data['living_area'] < 8))
            , 'total_area'
        ].apply(lambda x: x * living_part_median_by_type)

In [353]:
display(
    real_estate_data['living_area'].describe()
)

count    23699.000000
mean        34.579383
std         22.347244
min          7.410000
25%         19.000000
50%         30.000000
75%         42.100000
max        409.700000
Name: living_area, dtype: float64

❌ 'kitchen_area' — если студия то кухня `NaN` проверим, если да то сделаем 0

In [354]:
display(
    real_estate_data.loc[real_estate_data['kitchen_area'].isna()
        , 'living_area'
    ].count()
)

2278

In [355]:
display(
    real_estate_data.loc[
        (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['studio'])
        , 'living_area'
    ].count()
)

149

In [356]:
display(
    real_estate_data.loc[
        ~ (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['studio'])
        , 'living_area'
    ].count()
)

0

😏

In [357]:
display(
    real_estate_data.loc[
        (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['is_apartment'])
        , 'living_area'
    ].count()
)

15

In [358]:
display(
    real_estate_data.loc[
        ~ (real_estate_data['kitchen_area'].isna())
        & (real_estate_data['is_apartment'])
    ].sample(10)
)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_type
20368,8,9897258.0,85.2,2018-07-23T00:00:00,3,3.00,4,48.48,3,True,False,False,10.20,2,Санкт-Петербург,44870.0,20802.0,0.0,NaN,0.0,NaN,NaN,0
19017,12,4887000.0,52.4,2018-03-19T00:00:00,1,3.10,4,24.70,2,True,False,False,11.70,5,Санкт-Петербург,44870.0,20802.0,0.0,NaN,1.0,792.0,113.0,0
21244,10,5500000.0,58.0,2018-12-13T00:00:00,3,2.60,9,41.00,2,True,False,False,7.00,5,Санкт-Петербург,28632.0,13261.0,0.0,NaN,1.0,544.0,12.0,0
22869,0,15000000.0,25.0,2018-07-25T00:00:00,1,2.65,5,14.00,5,True,False,False,11.00,5,Санкт-Петербург,34963.0,8283.0,1.0,223.0,3.0,30.0,19.0,0
8611,6,6030000.0,50.4,2018-02-20T00:00:00,2,3.10,4,37.00,2,True,False,False,11.00,0,Санкт-Петербург,45767.0,21700.0,0.0,NaN,1.0,576.0,NaN,0
3067,13,6600000.0,95.0,2018-09-08T00:00:00,2,4.50,7,45.00,7,True,False,False,25.00,2,Выборг,NaN,NaN,NaN,NaN,NaN,NaN,96.0,0
5297,19,45000000.0,96.0,2018-11-15T00:00:00,2,3.20,6,54.72,4,True,False,False,45.00,1,Санкт-Петербург,21860.0,318.0,2.0,160.0,3.0,569.0,NaN,0
22938,14,4000000.0,98.0,2018-03-15T00:00:00,4,2.70,2,73.00,2,True,False,False,9.00,1,Нижняя,NaN,NaN,NaN,NaN,NaN,NaN,27.0,3
10390,18,30000000.0,137.0,2018-08-07T00:00:00,3,3.00,5,90.00,4,True,False,False,12.00,0,Санкт-Петербург,23832.0,1285.0,3.0,374.0,3.0,32.0,34.0,0
2975,9,4931398.0,42.9,2018-07-23T00:00:00,1,3.15,4,20.39,3,True,False,False,10.29,1,Санкт-Петербург,44870.0,20802.0,0.0,NaN,1.0,792.0,273.0,0


In [359]:
real_estate_data.loc[
        (real_estate_data['kitchen_area'].isna())
        & ((real_estate_data['studio'])
          | (real_estate_data['is_apartment']))
        , 'kitchen_area'
] = 0

In [360]:
types_na_kitchen_area = (
    list(real_estate_data.loc[
            (real_estate_data['kitchen_area'].isna())
            , 'locality_type'
        ].unique()
    )
)

for type_kitchen_area in types_na_kitchen_area:
    
    kitchen_area_part_mdn_by_type = round(pd.Series(
        real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & ~(real_estate_data['kitchen_area'].isna())
            , 'kitchen_area'
        ] / real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & ~(real_estate_data['kitchen_area'].isna())
            , 'total_area'
        ]
    ).median(), 2)
    
    if math.isnan(kitchen_area_part_mdn_by_type):
        print('NANы kitchen_area_part_mdn_by_type', type_kitchen_area)
        
    real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & (real_estate_data['kitchen_area'].isna())
            , 'kitchen_area'
    ] = real_estate_data.loc[
            (real_estate_data['locality_type'] == type_kitchen_area)
            & (real_estate_data['kitchen_area'].isna())
            , 'total_area'
        ].apply(lambda x: x * kitchen_area_part_mdn_by_type)

In [361]:
display(
    real_estate_data['kitchen_area'].describe()
)

count    23699.000000
mean        10.460818
std          6.062755
min          0.000000
25%          7.000000
50%          9.000000
75%         12.000000
max        112.000000
Name: kitchen_area, dtype: float64

❌ 'airports_nearest' — проверим по районам, может и нет ближайших аэропортов тогда `NaN` = 0

In [362]:
display(
    real_estate_data['airports_nearest'].describe()
)

count    18157.000000
mean     28793.672193
std      12630.880622
min          0.000000
25%      18585.000000
50%      26726.000000
75%      37273.000000
max      84869.000000
Name: airports_nearest, dtype: float64

In [363]:
display(
    real_estate_data['airports_nearest'].info()
)

<class 'pandas.core.series.Series'>
RangeIndex: 23699 entries, 0 to 23698
Series name: airports_nearest
Non-Null Count  Dtype  
--------------  -----  
18157 non-null  float64
dtypes: float64(1)
memory usage: 185.3 KB


None

In [364]:
display(
    real_estate_data.loc[
        real_estate_data['airports_nearest'].isna()
        , 'locality_type'
    ].unique()
)

[2, 1, 0, 3, 4, ..., 7, 8, 9, 10, 11]
Length: 12
Categories (12, int64): [0, 1, 2, 3, ..., 8, 9, 10, 11]

In [367]:
print(
    real_estate_data.loc[
        real_estate_data['airports_nearest'].isna()
        , 'locality_name'
    ].unique().isna()
)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [246]:
real_estate_data.loc[
    real_estate_data['airports_nearest'] <= 1000
] = np.nan


geo :)

In [ ]:
without_geo = {'Санкт-Петербург': 14315.0, 'Шушары': 12114.0, 'Янино-1': 35175.0, 'Парголово': 27207.0, 'Мурино': 30528.0, 'Ломоносов': 55794.0, 'Сертолово': 33605.0, 'Петергоф': 41071.0, 'Пушкин': 17850.0, 'Кудрово': 28907.0, 'Коммунар': 21903.0, 'Колпино': 36388.0, 'Красный Бор': 46244.0, 'Гатчина': 27642.0, 'Фёдоровское': 31725.0, 'Выборг': 190280.0, 'Кронштадт': 57870.0, 'Кировск': 79157.0, 'Новое Девяткино': 33673.0, 'Металлострой': 33632.0, 'Лебяжье': 85865.0, 'Сиверский': 48425.0, 'Молодцово': 92246.0, 'Кузьмоловский': 38805.0, 'Новая Ропша': 45108.0, 'Павловск': 21110.0, 'Пикколово': 17590.0, 'Всеволожск': 47554.0, 'Волхов': 231067.0, 'Кингисепп': 189494.0, 'Приозерск': 120272.0, 'Сестрорецк': 43533.0, 'Куттузи': 22949.0, 'Аннино': 24353.0, 'Ефимовский': 488800.0, 'Плодовое': 100751.0, 'Заклинье': 111442.0, 'Торковичи': 91185.0, 'Первомайское': 79893.0, 'Красное Село': 21392.0, 'Понтонный': 39635.0, 'Сясьстрой': 256016.0, 'Старая': 41209.0, 'Лесколово': 50172.0, 'Новый Свет': 24625.0, 'Сланцы': 250814.0, 'Путилово': 130547.0, 'Ивангород': 230997.0, 'Шлиссельбург': 85869.0, 'Никольское': 58207.0, 'Зеленогорск': 73307.0, 'Сосновый Бор': 130994.0, 'Оржицы': 62089.0, 'Кальтино': 46444.0, 'Романовка': 53296.0, 'Бугры': 29253.0, 'Рощино': 86119.0, 'Кириши': 195596.0, 'Луга': 112837.0, 'Волосово': 93509.0, 'Отрадное': 58156.0, 'Павлово': 67502.0, 'Оредеж': 94884.0, 'Копорье': 135738.0, 'Молодёжное': 97068.0, 'Тихвин': 363078.0, 'Победа': 106560.0, 'Нурма': 85336.0, 'Синявино': 104581.0, 'Тосно': 71468.0, 'Стрельна': 26735.0, 'Бокситогорск': 397866.0, 'Александровская': 197079.0, 'Лопухинка': 95449.0, 'Пикалёво': 434099.0, 'Терволово': 43181.0, 'Советский': 191244.0, 'Подпорожье': 444725.0, 'Петровское': 81636.0, 'Токсово': 43003.0, 'Сельцо': 113022.0, 'Вырица': 38084.0, 'Кипень': 47803.0, 'Келози': 50620.0, 'Вартемяги': 37207.0, 'Тельмана': 38545.0, 'Севастьяново': 132162.0, 'Большая Ижора': 78980.0, 'Агалатово': 40356.0, 'Новогорелово': 11587.0, 'Лесогорский': 196306.0, 'Лаголово': 29116.0, 'Цвелодубово': 97120.0, 'Рахья': 68928.0, 'Белогорка': 45992.0, 'Заводской': 55107.0, 'Новоселье': 21400.0, 'Большие Колпаны': 34343.0, 'Горбунки': 32156.0, 'Батово': 61927.0, 'Заневка': 30089.0, 'Иссад': 231778.0, 'Приморск': 191692.0, 'Мистолово': 34083.0, 'Новая Ладога': 228691.0, 'Зимитицы': 130114.0, 'Барышево': 114926.0, 'Разметелево': 45776.0, 'Свердлова': 40593.0, 'Пеники': 70569.0, 'Рябово': 101811.0, 'Пудомяги': 20058.0, 'Корнево': 58291.0, 'Низино': 43884.0, 'Бегуницы': 109020.0, 'Поляны': 123240.0, 'Мга': 87824.0, 'Елизаветино': 60776.0, 'Кузнечное': 134822.0, 'Колтуши': 44547.0, 'Запорожское': 79018.0, 'Гостилицы': 71972.0, 'Малое Карлино': 7278.0, 'Мичуринское': 85984.0, 'Морозова': 86398.0, 'Песочный': 88151.0, 'Сосново': 72957.0, 'Аро': 44388.0, 'Ильичёво': 73136.0, 'Тайцы': 21796.0, 'Малое Верево': 21382.0, 'Извара': 93307.0, 'Вещево': 145529.0, 'Паша': 310720.0, 'Калитино': 80290.0, 'Ульяновка': 85865.0, 'Чудской Бор': 134235.0, 'Дубровка': 73585.0, 'Мины': 38939.0, 'Войсковицы': 43548.0, 'Коркино': 113887.0, 'Ропша': 45740.0, 'Приладожский': 117497.0, 'Щеглово': 57841.0, 'Гаврилово': 156394.0, 'Лодейное Поле': 373503.0, 'Рабитицы': 104970.0, 'Никольский': 434204.0, 'Кузьмолово': 40057.0, 'Малые Колпаны': 33291.0, 'Петро-Славянка': 26751.0, 'Назия': 146004.0, 'Репино': 58800.0, 'Углово': 57897.0, 'Старая Малукса': 122944.0, 'Меньково': 44754.0, 'Старые Бегуницы': 106739.0, 'Сапёрный': 94240.0, 'Семрино': 27973.0, 'Глажево': 205228.0, 'Кобринское': 39560.0, 'Гарболово': 57689.0, 'Юкки': 30385.0, 'Приветнинское': 100485.0, 'Мануйлово': 162125.0, 'Пчева': 205365.0, 'Цвылёво': 325948.0, 'Мельниково': 122112.0, 'Пудость': 30912.0, 'Усть-Луга': 222531.0, 'Светогорск': 201116.0, 'Любань': 116705.0, 'Селезнёво': 205214.0, 'Каменногорск': 168411.0, 'Кривко': 75697.0, 'Глебычево': 183269.0, 'Парицы': 36148.0, 'Жилпосёлок': 562916.0, 'Войскорово': 33921.0, 'Стеклянный': 56440.0, 'Важины': 430627.0, 'Мыза-Ивановка': 28149.0, 'Русско-Высоцкое': 37727.0, 'Форносово': 38199.0, 'Старая Ладога': 225005.0, 'Житково': 135273.0, 'Виллози': 20780.0, 'Лампово': 53853.0, 'Шпаньково': 55595.0, 'Лаврики': 32934.0, 'Сумино': 92843.0, 'Возрождение': 177566.0, 'Старосиверская': 46211.0, 'Кикерино': 78050.0, 'Старое Хинколово': 50636.0, 'Пригородный': 28993.0, 'Торфяное': 107371.0, 'Будогощь': 246142.0, 'Суходолье': 92057.0, 'Красная Долина': 152144.0, 'Хапо-Ое': 53189.0, 'Дружная Горка': 52524.0, 'Лисий Нос': 35558.0, 'Яльгелево': 36992.0, 'Рождествено': 58092.0, 'Старополье': 200154.0, 'Левашово': 30104.0, 'Сяськелево': 53081.0, 'Камышовка': 146994.0, 'Лесная Поляна': 46710.0, 'Хязельки': 47880.0, 'Жилгородок': 42240.0, 'Ялгино': 340851.0, 'Новый Учхоз': 44620.0, 'Гончарово': 171460.0, 'Почап': 96293.0, 'Сапёрное': 94240.0, 'Платформа': 68493.0, 'Каложицы': 140887.0, 'Фалилеево': 150362.0, 'Пельгора': 112937.0, 'Торошковичи': 119943.0, 'Белоостров': 45330.0, 'Алексеевка': 169198.0, 'Серебрянский': 138481.0, 'Лукаши': 20202.0, 'Тарасово': 80842.0, 'Кингисеппский': 193006.0, 'Ушаки': 75003.0, 'Котлы': 168986.0, 'Сижно': 238349.0, 'Торосово': 82779.0, 'Форт Красная Горка': 104958.0, 'Новолисино': 59515.0, 'Громово': 86634.0, 'Глинка': 28120.0, 'Старая Пустошь': 49271.0, 'Коммунары': 108652.0, 'Починок': 113176.0, 'Вознесенье': 589006.0, 'Разбегаево': 37812.0, 'Гладкое': 66698.0, 'Тёсово-4': 101535.0, 'Бор': 355091.0, 'Коробицыно': 92007.0, 'Большая Вруда': 120252.0, 'Курковицы': 75929.0, 'Кобралово': 24984.0, 'Суоранда': 40220.0, 'Кондратьево': 247473.0, 'Счастье': 30682.0, 'Реброво': 258246.0, 'Тойворово': 50101.0, 'Семиозерье': 118854.0, 'Лесное': 56363.0, 'Совхозный': 446162.0, 'Ленинское': 62068.0, 'Суйда': 36242.0, 'Нижние Осельки': 44339.0, 'Свирь': 436789.0, 'Перово': 169523.0, 'Высоцк': 204984.0, 'Шум': 167635.0, 'Котельский': 169541.0, 'Лужайка': 220989.0, 'Большая Пустомержа': 158569.0, 'Красносельское': 112292.0, 'Вахнова Кара': 320440.0, 'Пижма': 30906.0, 'Кивеннапа Север': 77634.0, 'Ромашки': 102443.0, 'Каськово': 86794.0, 'Куровицы': 41973.0, 'Плоское': 100421.0, 'Кирпичное': 119228.0, 'Ям-Тесово': 93623.0, 'Раздолье': 75413.0, 'Терпилицы': 105090.0, 'Шугозеро': 436666.0, 'Ваганово': 87528.0, 'Пушное': 106081.0, 'Садко': 84687.0, 'Усть-Ижора': 37056.0, 'Выскатка': 243401.0, 'Свирьстрой': 394972.0, 'Кисельня': 208270.0, 'Трубников Бор': 134785.0, 'Высокоключевой': 36944.0, 'Пансионат Зелёный Бор': 116992.0, 'Ненимяки': 54384.0, 'Снегирёвка': 72608.0, 'Рапполово': 39316.0, 'Пустынка': 61528.0, nan: 85865.0, 'Большой Сабск': 154145.0, 'Русско': 212440.0, 'Лупполово': 33824.0, 'Большое Рейзино': 30860.0, 'Малая Романовка': 55468.0, 'Дружноселье': 51307.0, 'Пчевжа': 233813.0, 'Володарское': 130361.0, 'Нижняя': 22336.0, 'Тихковицы': 51005.0, 'Борисова Грива': 81305.0, 'Дзержинского': 117110.0}

In [275]:
real_estate_data.loc[
    real_estate_data['airports_nearest'].isna()
    , 'airports_nearest'
] = real_estate_data.loc[
        real_estate_data['airports_nearest'].isna()
        , 'locality_name'
    ].apply(lambda x:
                airprt_pulkovo_nrst.loc[
                    airprt_pulkovo_nrst['locality_name'] == x
                    , 'airprt_pulkovo'
                ].item()
            )

In [276]:
display(
    real_estate_data['airports_nearest'].describe()
)

count     23698.000000
mean      41771.048949
std       48154.229991
min        6450.000000
25%       20888.250000
50%       30528.000000
75%       42662.000000
max      589006.000000
Name: airports_nearest, dtype: float64

In [280]:
print(
    airprt_pulkovo_nrst.set_index('locality_name')['airprt_pulkovo'].to_dict()
)

{'Санкт-Петербург': 14315.0, 'Шушары': 12114.0, 'Янино-1': 35175.0, 'Парголово': 27207.0, 'Мурино': 30528.0, 'Ломоносов': 55794.0, 'Сертолово': 33605.0, 'Петергоф': 41071.0, 'Пушкин': 17850.0, 'Кудрово': 28907.0, 'Коммунар': 21903.0, 'Колпино': 36388.0, 'Красный Бор': 46244.0, 'Гатчина': 27642.0, 'Фёдоровское': 31725.0, 'Выборг': 190280.0, 'Кронштадт': 57870.0, 'Кировск': 79157.0, 'Новое Девяткино': 33673.0, 'Металлострой': 33632.0, 'Лебяжье': 85865.0, 'Сиверский': 48425.0, 'Молодцово': 92246.0, 'Кузьмоловский': 38805.0, 'Новая Ропша': 45108.0, 'Павловск': 21110.0, 'Пикколово': 17590.0, 'Всеволожск': 47554.0, 'Волхов': 231067.0, 'Кингисепп': 189494.0, 'Приозерск': 120272.0, 'Сестрорецк': 43533.0, 'Куттузи': 22949.0, 'Аннино': 24353.0, 'Ефимовский': 488800.0, 'Плодовое': 100751.0, 'Заклинье': 111442.0, 'Торковичи': 91185.0, 'Первомайское': 79893.0, 'Красное Село': 21392.0, 'Понтонный': 39635.0, 'Сясьстрой': 256016.0, 'Старая': 41209.0, 'Лесколово': 50172.0, 'Новый Свет': 24625.0, 'Сланц

In [ ]:
# real_estate_data = real_estate_data.astype(
#     {'airports_nearest': 'int32', }
# )

❌ 'city_centers_nearest' — проверим по районам, может и нет ближайших центров города тогда `NaN` = 0

In [369]:
display(
    len(real_estate_data.loc[
        ~real_estate_data['city_centers_nearest'].isna()
        , 'locality_name'
    ].unique())
)

27

In [368]:
display(
    len(
        real_estate_data.loc[
            real_estate_data['city_centers_nearest'].isna()
            , 'locality_name'
        ].unique()
    )
)

287

In [373]:
real_estate_data.loc[
    (real_estate_data['city_centers_nearest'].isna())
    & (real_estate_data['locality_name'] == 'Санкт-Петербург')
    , 'city_centers_nearest'
] = real_estate_data.loc[
        (~real_estate_data['city_centers_nearest'].isna())
        & (real_estate_data['locality_name'] == 'Санкт-Петербург')
        , 'city_centers_nearest'
    ].median()

In [376]:
columns = ('locality_name', 'locality_type', 'dist_to_center')
dist_to_hares_island = pd.DataFrame(columns=columns)

dist_to_hares_island['locality_name'] = (
    real_estate_data.loc[
        real_estate_data['city_centers_nearest'].isna()
        , 'locality_name'
    ].unique()
)
dist_to_hares_island.reset_index(drop=True, inplace=True)

# airprt_pulkovo_nrst['locality_type'] = (
#     airprt_pulkovo_nrst['locality_name']
#         .apply(lambda x:
#                locality_type_series.iloc[
#                    real_estate_data.loc[
#                        real_estate_data['locality_name'] == x
#                        , 'locality_type'
#                    ].sample(1).item()
#                ]
#                )
# )

dist_to_hares_island['locality_type'] = (
    dist_to_hares_island['locality_name']
        .apply(lambda z: locality_type_series.iloc[
                real_estate_data.loc[
                    real_estate_data['locality_name'] == z
                    , 'locality_type'
                ].sample(1).item()
            ]
        )
)

display(
    dist_to_hares_island.sample(7)
)

,locality_name,locality_type,dist_to_center
283,Тихковицы,деревня,NaN
120,Дубровка,посёлок городского типа,NaN
63,Токсово,посёлок городского типа,NaN
173,Старосиверская,деревня,NaN
275,Русско,деревня,NaN
212,Форт Красная Горка,посёлок,NaN
282,Нижняя,деревня,NaN


In [377]:
def to_hares_island(
        locality_name,
        locality_type,
        region='ленинградская область'):

    address = (region, locality_type, locality_name)
    address = ', '.join(address)
    maps_url = 'https://geocode-maps.yandex.ru/1.x/'
    keys = {'apikey': api_key,
            'geocode': address,
            'format': 'json',
            'results': '1',
            }

    row_data = requests.get(maps_url, params=keys)
    parsed = json.loads(row_data.text)
    locality_position = (json.dumps(
        parsed['response']['GeoObjectCollection']
        ['featureMember'][0]['GeoObject']['Point']['pos']
        , indent=4, ensure_ascii=False
    )
    )
    locality_position = locality_position.replace('\"', '')
    long, lat = locality_position.split(' ')
    locality_long_lat = (float(long), float(lat))
    dist_in_m = distance.distance(
        locality_long_lat,
        hares_island_long_lat
    ).m
    return (float(round(dist_in_m)))

In [379]:
dist_to_hares_island['dist_to_center'] = dist_to_hares_island.apply(
    lambda x: to_hares_island(locality_type=str(x[1]),  locality_name=str(x[0]))
    ,axis=1
)

In [380]:
display(
    dist_to_hares_island.sample(7)
)

,locality_name,locality_type,dist_to_center
198,Каложицы,посёлок,150757.0
150,Светогорск,город,196810.0
271,Рапполово,деревня,23850.0
185,Старополье,деревня,211153.0
239,Свирь,посёлок станции,427886.0
207,Кингисеппский,посёлок,201975.0
167,Виллози,городской посёлок,34230.0


In [393]:
dist_to_hares_island.set_index('locality_name', inplace=True)
display(dist_to_hares_island)

,locality_type,dist_to_center
locality_name,,
Янино-1,городской посёлок,26450.0
Мурино,посёлок,16121.0
Сертолово,город,21857.0
Кудрово,город,21635.0
Коммунар,город,32655.0
...,...,...
Володарское,посёлок,145979.0
Нижняя,деревня,36037.0
Тихковицы,деревня,65762.0


In [391]:
# dist_to_hares_island.reset_index(inplace=True)
# display(dist_to_hares_island)

,locality_name,locality_type,dist_to_center
0,Янино-1,городской посёлок,26450.0
1,Мурино,посёлок,16121.0
2,Сертолово,город,21857.0
3,Кудрово,город,21635.0
4,Коммунар,город,32655.0
...,...,...,...
281,Володарское,посёлок,145979.0
282,Нижняя,деревня,36037.0
283,Тихковицы,деревня,65762.0
284,Борисова Грива,деревня,71880.0


In [394]:
real_estate_data.loc[
    real_estate_data['city_centers_nearest'].isna()
    , 'city_centers_nearest'
] = real_estate_data.loc[
        real_estate_data['city_centers_nearest'].isna()
        , 'locality_name'
    ].apply(lambda x:
                dist_to_hares_island.loc[
                    dist_to_hares_island['locality_name'] == x
                    , 'dist_to_center'
                ].item()
            )

display(dist_to_hares_island)

KeyError: 'locality_name'

Так, похоже, что у всех `'locality_name'` = `NaN`, `'is_apartment'` = `False`

`ceiling_height` — пропуски заменим медианой по району `locality_name`

Приведите данные к нужным типам:
Поясните, в каких столбцах нужно изменить тип данных и почему.

## Шаг 3. Посчитайте и добавьте в таблицу

цену квадратного метра;
день недели, месяц и год публикации объявления;
этаж квартиры; варианты — первый, последний, другой;
соотношение жилой и общей площади, а также отношение площади кухни к общей.

## Шаг 4. Проведите исследовательский анализ данных

Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.
Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.
Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.
Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите среди них населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце locality_name.
Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (locality_name). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.
Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

## Шаг 5. Напишите общий вывод

## Описание данных

`airports_nearest` — расстояние до ближайшего аэропорта в метрах (м)

`balcony` — число балконов

`ceiling_height` — высота потолков (м)

`cityCenters_nearest` — расстояние до центра города (м)

`days_exposition` — сколько дней было размещено объявление (от публикации до снятия)

`first_day_exposition` — дата публикации

`floor` — этаж

`floors_total` — всего этажей в доме

`is_apartment` — апартаменты (булев тип)

`kitchen_area` — площадь кухни в квадратных метрах (м²)

`last_price` — цена на момент снятия с публикации

`living_area` — жилая площадь в квадратных метрах (м²)

`locality_name` — название населённого пункта

`open_plan` — свободная планировка (булев тип)

`parks_around3000` — число парков в радиусе 3 км

`parks_nearest` — расстояние до ближайшего парка (м)

`ponds_around3000` — число водоёмов в радиусе 3 км

`ponds_nearest` — расстояние до ближайшего водоёма (м)

`rooms` — число комнат

`studio` — квартира-студия (булев тип)

`total_area` — площадь квартиры в квадратных метрах (м²)

`total_images` — число фотографий квартиры в объявлении